In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import os
import time
from selenium.webdriver.common.by import By
import re
from timeit import default_timer as timer
from difflib import SequenceMatcher
import pandas as pd

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

options = Options()
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--headless")
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--no-sandbox")  # Bypass OS security model
options.add_argument("--disable-dev-shm-usage")  # Overcome resource problems
options.add_argument("--disable-gpu")  # Disable GPU
options.add_argument("--window-size=400,400")  # Set a fixed window size (optional but helps)
options.add_argument("--disable-software-rasterizer")  # Disable rendering components
options.add_argument("--hide-scrollbars")  # Hide scrollbars
print("Done")

Done


In [3]:
def search(card, options):
    driver = webdriver.Chrome(options = options)
    driver.get("https://tagger.scryfall.com/")
    search_box = driver.find_element(By.CLASS_NAME, "tag-input-field")
    search_box.send_keys(card)
    time.sleep(2)

    selecter = driver.find_element(By.CLASS_NAME, "suggestion-menu") #Finds the dropdown of suggestions
    time.sleep(2)

    search = selecter.find_elements(By.CLASS_NAME, "suggestion-row") #Pulls all of the rows of the dropdown
    searchlist = []
    for i in search:
        searchlist.append(i.get_attribute("innerHTML")) 
    cardstrings = []
    similarlist = []
 
    for i in searchlist: 
        if card in i:
            if "#icons-card" in i: #Find the value of the dropdown that is a specific card
                i = i.split(">")
                for j in i:
                    if card in j:
                        cardstrings.append(j.replace(" <!----",""))
    
    for i in cardstrings:
        sim = similar(card, i)
        similarlist.append(sim)

    
    cardsearch = similarlist.index(max(similarlist))

    if "#icons-tag" in searchlist[cardsearch+1]:
        selecter.find_elements(By.CLASS_NAME, "suggestion-row")[cardsearch + 2].click()
    else:
        selecter.find_elements(By.CLASS_NAME, "suggestion-row")[cardsearch + 1].click()
    time.sleep(1)


    try:
        specific = driver.find_element(By.CLASS_NAME, "card-grid-item")
        specific.click()
    except:
        pass

    time.sleep(2)
    tags = driver.find_elements(By.CLASS_NAME, "tag-row-flex")

    taglist = []
    for i in tags:
        taglist.append(i.get_attribute("innerHTML"))

    cardtags = []
    for i in taglist:
        if "tags/card/" in i:
            cardtags.append(i)
    
    tagsavelist = []
    for i in cardtags:
        if "tags/card/" in i:
            tag = i.split('"">')[-1].replace("</a>","")
            tagsavelist.append(tag)

    time.sleep(2)
    oldtags = driver.find_elements(By.CLASS_NAME, "tagging-ancestors")

    taglist = []
    for i in oldtags:
        taglist.append(i.get_attribute("innerHTML"))

    cardtags = []
    for i in taglist:
        if "tags/card/" in i:
            cardtags.append(i)

    for i in cardtags:
        if "/tags/card/" in i:
            tag = i.split(",")
            for k in tag:
                tagsavelist.append(k.split('"">')[-1].replace("</a>",""))
            

    return tagsavelist
    

card = "Kianne, Dean of Substance // Imbraham, Dean of Theory"
searchlist = search(card, options)
searchlist

['cycle-stx-dean',
 'evasion',
 'gives pp counters',
 'mana sink',
 'repeatable token generator',
 'cycle']

In [4]:
dataset = pd.read_csv("updated-cardlist.csv")
if "Unnamed: 0" in dataset.columns:
    dataset.drop("Unnamed: 0", axis=1, inplace=True)
totalcards = dataset["cardname"]
tagdict = {}
errorlist = []

In [5]:
count = 0
done = pd.read_csv("scryfall_taglist.csv")
if "Unnamed: 0" in done.columns:
    done.drop("Unnamed: 0", axis=1, inplace=True)
for i in totalcards:
    if i in list(done["Card"]):
        continue
    else:
        try:
            tagdict[i] = search(i, options)
        except:
            tagdict[i] = []
            errorlist.append(i)
            print("----- Error in finding tags for", i)
            continue
        
        if count % 10 == 0:
            print(i)

        if count % 100 == 0:
            newdf = pd.DataFrame(list(tagdict.items()), columns=['Card', 'Tags'])
            df = pd.concat([done, newdf], axis=0)
            df.reset_index(inplace = True)
            if "index" in df.columns:
                df.drop("index", axis = 1, inplace = True)
            df.to_csv("scryfall_taglist.csv")
            print("---- Saved to csv ----")
        count += 1


Broadside Bombardiers
---- Saved to csv ----
Don Andres, the Renegade
Master of Dark Rites
Scion of Calamity
Wave Goodbye
Counterpoint
Hot Pursuit
Nelly Borca, Impulsive Accuser
Sophia, Dogged Detective
Absolving Lammasu
Anzrag's Rampage
---- Saved to csv ----
Basilica Stalker
Buried in the Garden
Case of the Pilfered Proof
Clandestine Meddler
Crime Novelist
Deduce
Due Diligence
Extract a Confession
Forensic Researcher
Gleaming Geardrake
---- Saved to csv ----
Hedge Maze
Innocent Bystander
Kellan, Inquisitive Prodigy // Tail the Suspect
Lead Pipe
Magnetic Snuffler
Neighborhood Guardian
On the Job
Private Eye
Reckless Detective
Rubblebelt Maverick
---- Saved to csv ----
Sharp-Eyed Rookie
Surveillance Monitor
Tolsimir, Midnight's Light
Undercity Sewers
Vengeful Tracker
Afterlife Insurance
Corporeal Projection
----- Error in finding tags for Dimir Strandcatcher
Incriminating Impetus
Resonance Technician
Tribune of Rot
---- Saved to csv ----
Aradesh, the Founder
Boomer Scrapper
Cass, Hand 

In [13]:
done = pd.read_csv("scryfall_taglist.csv")
if "level_0" in done.columns:
    done.drop("level_0", axis=1, inplace=True)
need2fix = done[done["Tags"] == "[]"]
if "Unnamed: 0" in need2fix.columns:
    need2fix.drop("Unnamed: 0", axis=1, inplace=True)
need2fix.to_csv("need2fix.csv")

C:\Users\josep\AppData\Local\Temp\ipykernel_12904\181829258.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need2fix.drop("Unnamed: 0", axis=1, inplace=True)


In [14]:
need2fix

,Card,Tags
29,Circle of Protection: Blue,[]
30,Circle of Protection: Green,[]
31,Circle of Protection: Red,[]
32,Circle of Protection: White,[]
75,Fire Elemental,[]
...,...,...
27342,Season of Gathering,[]
27343,Season of Loss,[]
27344,Season of the Bold,[]
27345,Season of the Burrow,[]
